In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
# from nltk.lm import Vocabulary
# vocabulary = Vocabulary(text)
# len(vocabulary)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 17
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])

556949
556932
['\ufeffThe Project Gute', 'The Project Guten', 'he Project Gutenb', 'e Project Gutenbe', ' Project Gutenber']


# fit

In [8]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(vocabulary)
net.to(device)

encoded = net.text2tensor(lines).to(device)
print(encoded.shape)

performance = net.fit(encoded, save_path = "checkpoints/autoregressive_lstm.pt", progress_bar = 0)
net.save_architecture("models/autoregressive_lstm.arch")

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

torch.Size([556932, 19])
Training started
Epochs: 5
Learning rate: 0.001
Weight decay: 0
Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------
    1 |   1.3814 |     37.656 |     0.9
    2 |   1.0845 |     31.058 |     1.7
    3 |   1.0266 |     29.660 |     2.6
    4 |   0.9955 |     28.889 |     3.4
    5 |   0.9748 |     28.380 |     4.3


FileNotFoundError: [Errno 2] No such file or directory: 'architectures/autoregressive_lstm.architecture'

In [ ]:
# includes all the information about the epoch and the model, useful for reproducibility

performance

In [ ]:
# the input for testing

net.tensor2text(encoded[:5])

# predict

In [10]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("models/autoregressive_lstm.arch")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
idx, log_probabilities = net.predict(encoded[:5], main_progress_bar = False, progress_bar = 0)

net.tensor2text(idx)

UnpicklingError: invalid load key, 'v'.

In [ ]:
log_probabilities

# greedy_search

In [ ]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("models/autoregressive_lstm.arch")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

# sample

In [ ]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("models/autoregressive_lstm.arch")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

# beam_search

In [ ]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("models/autoregressive_lstm.arch")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5], progress_bar = False)

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])